In [ ]:
import os
import sys
sys.path.insert(0, "../..")
sys.path.insert(0, "../../../acquire")
import ipywidgets as widgets

from HUGS.Processing import search


from Acquire.Client import User, Drive, Service, PAR, Authorisation, StorageCreds

## Start with uploading and processing of the data

In [ ]:
base_url = "https://hugs.acquire-aaai.com/t"
hugs_url = "https://hugs.acquire-aaai.com"
user = User(username="gareth", identity_url="%s/identity" % base_url)
response = user.request_login()

Check we're logged in

In [ ]:
user.wait_for_login()

### Upload a file for processing


In [ ]:
filename = "bsd.picarro.1minute.248m.dat"
dir_path = os.path.abspath("")
test_data = "../../test/data/proc_test_data/CRDS"
filepath = os.path.join(dir_path, test_data, filename)

In [ ]:
creds = StorageCreds(user=user, service_url="%s/storage" % base_url)
drive = Drive(creds=creds, name="test_drive")
filemeta = drive.upload(filepath)
par = PAR(location=filemeta.location(), user=user)

In [ ]:
hugs = Service(service_url="%s/hugs" % base_url)
par_secret = hugs.encrypt_data(par.secret())
auth = Authorisation(resource="process", user=user)

In [ ]:
hugs

In [ ]:
args = {"authorisation": auth.to_data(),
            "par": {"data": par.to_data()},
            "par_secret": {"data": par_secret},
            "data_type": "CRDS"}

response = hugs.call_function(function="process", args=args)

In [ ]:
response

In [ ]:
from HUGS.Processing import search

search_terms = widgets.Text(
                    value="Search",
                    placeholder = "Search",
                    description = "Search terms:",
                    disabled = False
)

locations = widgets.Text(
                value="Locations",
                placeholder = "BSD, HFD",
                description = "Locations:",
                disabled = False
)

data_type = widgets.Dropdown(
                options=["CRDS", "GC"], 
                value="CRDS",
                description="Data type",
                disabled=False
    )



layout=widgets.Layout(display = "flex", width = "50%")
search_button = widgets.Button(description="Search", button_style="success", \
                               layout=widgets.Layout(flex='1 1 0%', width='25%'))

box = widgets.VBox(children=[search_terms, locations, data_type, search_button], layout=layout)

def call_search(x):
    """ Call the search function and pass it the values 
        in the text boxes
            
    """
    x =  search(search_terms=search_terms.value, locations=locations.value, data_type=data_type.value)
    print(x)
    



In [ ]:
search_button.on_click(call_search)

In [ ]:
box